In [1]:
# 基础的asm文件，用于构建CFG
fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/staticAnalysis/hw-memory/inputs/spce_changes/small/small.asm"

# 带bss段和data段的asm文件，用于查看全局变量地址空间
seg_fp = r'/Users/gugujixiao/workspace/project/HWMemory/Code/staticAnalysis/hw-memory/inputs/spce_changes/small/smallD.asm'

# 带源码的asm文件，用于源码和汇编代码对照
code_fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/staticAnalysis/hw-memory/inputs/spce_changes/small/smallL.asm"


In [2]:
'''读指令, asm文件对象化'''
from cfg.cfg import proc_identify
from cfg.isa import Instruction, AddrMode
from typing import Tuple
from cfg.read_asm import AsmFileReader, StatementType
from cache_analysis.read_segment import segmentReader
from cfg.read_codeMix import AsmCodeReader, MixStatementType

reader =AsmFileReader(fp)#asm对象
segreader = segmentReader(seg_fp)#带bss和data段的asm对象
codereader = AsmCodeReader(code_fp)#带源码对照的asm对象

statements = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

procs = proc_identify(statements)

# for proc in procs:
#     print(proc.name, proc.beg_addr.hex_str())
#     if False and proc.name == 'main':
#         for inst in proc.instruction:
#             print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
# print()

In [3]:
'''CFG:分proc'''
from graphviz import Digraph
from cfg.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output/procedures.svg'

In [4]:
'''CFG:处理call_graph'''
from cfg.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output/call_graph.svg'

In [5]:
'''CFG:生成TCFG和loop信息'''
from cfg.cfg import TCfg


tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()
tcfg.build_loop_hrchy()


#for l in tcfg.loops:
#    print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))
#    l.back_edge.is_backEdge = True
#for n in tcfg.all_nodes:
#     temp = n.inside_loop.name if n.inside_loop is not None else 'None'
#     print("{} {}".format(n.name, temp))



#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




In [6]:
'''生成插桩相关信息'''
# 生成loop中各node首尾地址
loop_range =list()

with open("./output/loop_addr.txt","w") as f:
    for l in tcfg.loops:
        #f.write(l.name+"\n")
        #print(l.name)
        for n in l.all_nodes:
            ins = n.instructions
            #head = ins[0].addr
            head,end = n.inst_range
            string = l.name[1:]+" "+head.hex_str()+"-"+end.hex_str()+"\n"
            loop_range.append(([head.val(),end.val()],l.name))
            f.write(string)
        f.write("\n")

# 生成源代码对于loop的对应关系
code_stat = list()# 中间变量
loop_code_range =list()# 最终输出


for i in range(len(codereader.statements)):
    s: Tuple[MixStatementType, tuple]
    if codereader.statements[i][0] == MixStatementType.codeline:
        codeline = codereader.statements[i][1]
        addr = ""
        not_find = True
        k = i
        while not_find:
            k += 1
            if codereader.statements[k][0] == MixStatementType.asm:
                addr =  int(codereader.statements[k][1][0],16) 
                not_find = False

        code_stat.append((addr,codeline,"loopname"))


with open("./output/loop_code.txt","w") as f:
    for i in code_stat:
        for l in loop_range:
            if l[0][0]<i[0]<l[0][1]:
                string = i[1]+l[1]
                loop_code_range.append((i[1],l[1]))
                f.write(string)
                f.write("\n")  


for i in loop_code_range:
    print(i)

('/home/guguji/big/main.c:159', 'l74')
('/home/guguji/big/main.c:159', 'l62')
('/home/guguji/big/main.c:160', 'l74')
('/home/guguji/big/main.c:160', 'l62')
('/home/guguji/big/main.c:176', 'l61')
('/home/guguji/big/main.c:176', 'l50')
('/home/guguji/big/main.c:177', 'l61')
('/home/guguji/big/main.c:177', 'l50')
('/home/guguji/big/main.c:218 (discriminator 3)', 'l2')
('/home/guguji/big/main.c:218 (discriminator 3)', 'l1')
('/home/guguji/big/main.c:218 (discriminator 3)', 'l0')
('/home/guguji/big/main.c:215 (discriminator 3)', 'l2')
('/home/guguji/big/main.c:215 (discriminator 3)', 'l1')
('/home/guguji/big/main.c:215 (discriminator 3)', 'l0')
('/home/guguji/big/main.c:235 (discriminator 3)', 'l6')
('/home/guguji/big/main.c:235 (discriminator 3)', 'l19')
('/home/guguji/big/main.c:235 (discriminator 3)', 'l38')
('/home/guguji/big/main.c:235 (discriminator 3)', 'l3')
('/home/guguji/big/main.c:235 (discriminator 3)', 'l13')
('/home/guguji/big/main.c:235 (discriminator 3)', 'l29')
('/home/gugu

In [7]:
from rw_analysis.rw_core import loadstore_Obj
lds_obj = loadstore_Obj(segreader,tcfg)

#用于给后面解决兼容性问题的数据
lsproc = lds_obj.lsproc
ls_loop_info = lds_obj.loop_info
lds_table = lds_obj.lsproc.ls_table

for i in lds_table:
    if i.target_num == 1:
        print("single",i.ins.tokens,i.reg_target,i.final_addr,i.addr_offset,"是否找到",i.is_find,i.node.name) 
    else:
        print("muti",i.ins.tokens,i.reg_target_list[0],i.reg_target_list[1],i.addr_offset,i.final_addr,"是否找到",i.is_find,i.node.name)


#for k,v in ls_loop_info.items():
#    print(k,v)

single ('4006cc', 'b9001fe0', 'str', None, 'w0, [sp, #28]') sp 0 28 是否找到 True n0
single ('4006d0', 'f9000be1', 'str', None, 'x1, [sp, #16]') sp 0 16 是否找到 True n0
single ('4006dc', 'b900001f', 'str', None, 'wzr, [x0]') 6fc000 7325440 1792 是否找到 True n0
single ('4006e4', 'b9002fe0', 'str', None, 'w0, [sp, #44]') sp 0 44 是否找到 True n0
single ('4006e8', 'b9002bff', 'str', None, 'wzr, [sp, #40]') sp 0 40 是否找到 True n0
single ('4006ec', 'b9402fe0', 'ldr', None, 'w0, [sp, #44]') sp 0 44 是否找到 True n0
single ('400718', 'b9000001', 'str', None, 'w1, [x0]') 6fc000 7325440 1792 是否找到 True n5
single ('40071c', 'b9402be0', 'ldr', None, 'w0, [sp, #40]') sp 0 40 是否找到 True n5
single ('400738', 'b9000001', 'str', None, 'w1, [x0]') 6fc000 7325440 1792 是否找到 True n7
single ('400754', 'b9000001', 'str', None, 'w1, [x0]') 6fc000 7325444 1796 是否找到 True n9
single ('40076c', 'b9000001', 'str', None, 'w1, [x0]') 6fc000 7325440 1792 是否找到 True n11
single ('400788', 'b9000001', 'str', None, 'w1, [x0]') 6fc000 7325444 1

In [8]:
# from newCFG.heat_analysis import loop_heat
# test=loop_heat(tcfg,lsproc,r'C:\Users\13377\Desktop\hw-memory-master (5)\hw-memory-master\benchmarks\final_benchmark\spec_benchD.asm')
# test.do_it()


In [9]:

# TODO.
from cache_analysis.cache_risk_level import CacheRisk

cache_test = CacheRisk(tcfg, lsproc, r'D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmarkD.asm',
                       r'D:\workspace\hw-memory\cache_analysis\new_cache\input\cache_information.in')
cache_test.test()


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\workspace\\hw-memory\\benchmarks\\final_benchmark\\spec_benchmarkD.asm'

In [ ]:

# from newCFG.rw_condition import RWProc, RWType
# from newCFG.rw_condition_out import RWOut_Proc



# rwout = RWOut_Proc(tcfg_nodes,segreader,rwproc.rw_table,tcfg.loops)




